In [6]:
using ITensors,ITensorMPS
using PastaQ
using RandomMeas
using ProgressBars


In [28]:
N = 8
ξ = siteinds("Qubit", N)
B = 1.
ampo = AutoMPO()
for j in 1:(N - 1)
  # Ising ZZ interactions
  ampo .+= -1, "X", j, "X", j + 1
end
for j in 1:N
  # Transverse field X
  ampo .+= -B, "Z", j
end
H = MPO(ampo,ξ)
H2 = apply(H,H)
# Density-matrix renormalization group
dmrg_iter = 5      # DMRG steps
dmrg_cutoff = 1E-10   # Cutoff
ψ0 = random_mps(ξ) # Initial state
sweeps = Sweeps(dmrg_iter)
maxdim!(sweeps, 10, 20, 30, 40, 50, 100)
cutoff!(sweeps, dmrg_cutoff)
# Run 
println("Running DMRG to get ground state of transverse field Ising model:")
E, ψ = dmrg(H, ψ0, sweeps)
println("\nGround state energy:  ", E)
println("\n---------------------------------------\n")

Running DMRG to get ground state of transverse field Ising model:
After sweep 1 energy=-9.804104279631897  maxlinkdim=4 maxerr=1.46E-16 time=0.008
After sweep 2 energy=-9.83786582249718  maxlinkdim=12 maxerr=2.77E-12 time=0.009
After sweep 3 energy=-9.837951443164044  maxlinkdim=8 maxerr=6.81E-11 time=0.008
After sweep 4 energy=-9.83795144619894  maxlinkdim=6 maxerr=6.91E-11 time=0.019
After sweep 5 energy=-9.837951446199057  maxlinkdim=6 maxerr=6.91E-11 time=0.012

Ground state energy:  -9.837951446199057

---------------------------------------



In [29]:
η = 10
m = 4
# Build the gate structure
circuit = []
for d in 1:η
    xx_layer = [("Rxx", (j, j + 1), (ϕ=1*d/η/m,)) for j in 1:(N - 1)]
    z_layer = [("Rz", j, (θ=2*B/m,)) for j in 1:N]
    for t in 1:m
        append!(circuit, xx_layer)
        append!(circuit, z_layer)
    end
end
ψ0 = MPS(ξ,["Up" for n in 1:N]);

In [30]:
ψt = runcircuit(ψ0, circuit; cutoff=1E-8)
orthogonalize!(ψt,1)
ψt[1] /= norm(ψt[1])
E_th = real(inner(ψt', H,ψt))
println("final energy ",E_th)
E2_th = real(inner(ψt', H2,ψt))-E_th^2
println("final energy variance ",E2_th)

final energy -9.47182617618306
final energy variance 1.6950453000525698


In [32]:
Nu = 400
NM = 1000

1000

In [33]:
using Statistics
Es = 0
E2s = 0
for r in ProgressBar(1:Nu, printing_delay=5)
        u = get_rotations(ξ,1)
        data = get_RandomMeas_MPS(ψt,u,NM)
        for m in 1:NM
            ρ = get_shadow_factorized(data[m,:],ξ,u)
            Es += get_expect_shadow(H,ρ,ξ)/NM/Nu
            E2s += get_expect_shadow(H2,ρ,ξ)/NM/Nu
        end
end
E2s = E2s .- Es.^2
println("estimated energy ",Es)
println("estimated energy variance ",E2s)

0.0%┣                                              ┫ 0/400 [00:00<00:00, -0s/it]
3.8%┣█▊                                            ┫ 15/400 [00:05<02:25, 3it/s]
7.5%┣███▌                                          ┫ 30/400 [00:10<02:12, 3it/s]
11.5%┣█████▏                                       ┫ 46/400 [00:16<02:04, 3it/s]
15.5%┣███████                                      ┫ 62/400 [00:21<01:55, 3it/s]
19.5%┣████████▊                                    ┫ 78/400 [00:26<01:49, 3it/s]
23.5%┣██████████▋                                  ┫ 94/400 [00:31<01:43, 3it/s]
27.5%┣████████████                                ┫ 110/400 [00:36<01:37, 3it/s]
31.2%┣█████████████▊                              ┫ 125/400 [00:41<01:32, 3it/s]
35.2%┣███████████████▌                            ┫ 141/400 [00:47<01:26, 3it/s]
39.0%┣█████████████████▏                          ┫ 156/400 [00:52<01:21, 3it/s]
42.8%┣██████████████████▉                         ┫ 171/400 [00:57<01:16, 3it/s]
46.8%┣████████████████████▋ 

estimated energy -9.492256466395915
estimated energy variance 1.8095465259972912


100.0%┣███████████████████████████████████████████┫ 400/400 [02:12<00:00, 3it/s]
100.0%┣███████████████████████████████████████████┫ 400/400 [02:12<00:00, 3it/s]
